In [1]:
import gym

In [2]:
# Descobrindo o escopo de ações que um agente pode ter no ambiente
env = gym.make('BreakoutNoFrameskip-v4') 
print(env.action_space)

Discrete(4)


In [3]:
# Definições das ações
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT']

In [4]:
# Gerando ações aleatórias
for i in range(10):
    print(env.action_space.sample())

2
2
0
1
2
3
2
3
0
2


In [5]:
# Visualizando um estado
env = gym.make('BreakoutNoFrameskip-v4') # cria um ambiente
obs = env.reset() 
print(obs)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [6]:
obs.shape

(210, 160, 3)

In [7]:
# Visualizando o agente interagindo com o ambiente
import time

env = gym.make('BreakoutNoFrameskip-v4')
obs = env.reset() 

for i in range(2000):
    env.render() 
    time.sleep(0.02)
    acao = [env.action_space.sample()] 
    obs, recompensa, terminou, info = env.step(acao)  
    if terminou: 
        print("Episódio finalizado depois de {} timesteps".format(i+1))
        break      
env.close()

C:\Users\Falcone\anaconda3\envs\gymoa\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Não é possível alterar o modo de thread depois de o mesmo estar definido
  warnings.warn(str(err))


Episódio finalizado depois de 490 timesteps


In [13]:
from OpenAIWrappers import wrapper
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common import set_global_seeds
from stable_baselines import PPO2
from stable_baselines.common.callbacks import CheckpointCallback

def cria_env(nome_env, num, seed=0):
    def _init():
        env = gym.make(nome_env)
        env = wrapper(env)
        env.seed(seed + num)
        return env
    set_global_seeds(seed)
    return _init

import time

num_cpu = 4 #numero de nucleos do meu computador laptop
#Google search: https://www.google.com/search?q=como+descobrir+a+quantidade+de+nucleos+do+processador+do+seu+computador&rlz=1C1GCEA_enBR904BR904&oq=como+descobrir+a+quantidade+de+nucleos+do+processador+do+seu+computador&aqs=chrome..69i57.11389j0j7&sourceid=chrome&ie=UTF-8
#Pressione Ctrl + Shift + ESC para abrir o Gerenciador de tarefas. Selecione a guia desempenho para ver quantos núcleos e processadores lógicos seu computador tem.
env = SubprocVecEnv([cria_env(nome_env='BreakoutNoFrameskip-v4', num=i) for i in range(num_cpu)])

checkpoint_callback = CheckpointCallback(save_freq=250000, save_path='C:/Users/Falcone/Documents/0_MachineLearning/2022.09.21_AprendizadoPorReforco/4-RodandoVariosAmbientesEmParalelo/', name_prefix='PPO')
# O parâmetro save_freq é contabilizado com base na quantidade de vezes que todos os envs foram chamados de forma conjunta. Por
# exemplo: se há 24 envs, uma chamada de step() aos 24 envs contabiliza 1. Como há 24 steps sendo realizados em uma chamada, 
# devemos colocar em save_freq o valor de n_steps/n_envs.


modelo = PPO2(policy='CnnPolicy', env=env, gamma=0.99, n_steps=128, learning_rate=0.00025, vf_coef=0.5, verbose=1)

inicio = time.time()
modelo.learn(total_timesteps=4000000, log_interval=10, callback=checkpoint_callback)
fim = time.time()
print('Tempo em horas:', (fim-inicio)/3600)

--------------------------------------
| approxkl           | 0.008582072   |
| clipfrac           | 0.08203125    |
| explained_variance | -0.576        |
| fps                | 196           |
| n_updates          | 1             |
| policy_entropy     | 1.378114      |
| policy_loss        | -0.0051042656 |
| serial_timesteps   | 128           |
| time_elapsed       | 0             |
| total_timesteps    | 512           |
| value_loss         | 0.022374783   |
--------------------------------------
-------------------------------------
| approxkl           | 0.006112767  |
| clipfrac           | 0.08935547   |
| explained_variance | 0.446        |
| fps                | 216          |
| n_updates          | 10           |
| policy_entropy     | 1.3443102    |
| policy_loss        | -0.008873755 |
| serial_timesteps   | 1280         |
| time_elapsed       | 21.4         |
| total_timesteps    | 5120         |
| value_loss         | 0.009371219  |
-------------------------------------

-------------------------------------
| approxkl           | 0.0018116264 |
| clipfrac           | 0.0024414062 |
| explained_variance | 0.882        |
| fps                | 225          |
| n_updates          | 170          |
| policy_entropy     | 1.2057513    |
| policy_loss        | -0.004529323 |
| serial_timesteps   | 21760        |
| time_elapsed       | 413          |
| total_timesteps    | 87040        |
| value_loss         | 0.008760022  |
-------------------------------------
--------------------------------------
| approxkl           | 0.00052694365 |
| clipfrac           | 0.0           |
| explained_variance | 0.944         |
| fps                | 216           |
| n_updates          | 180           |
| policy_entropy     | 1.153566      |
| policy_loss        | -0.0017897963 |
| serial_timesteps   | 23040         |
| time_elapsed       | 436           |
| total_timesteps    | 92160         |
| value_loss         | 0.00604813    |
--------------------------------------

-------------------------------------
| approxkl           | 0.0029006398 |
| clipfrac           | 0.032226562  |
| explained_variance | 0.92         |
| fps                | 207          |
| n_updates          | 340          |
| policy_entropy     | 0.8651529    |
| policy_loss        | -0.005018134 |
| serial_timesteps   | 43520        |
| time_elapsed       | 826          |
| total_timesteps    | 174080       |
| value_loss         | 0.0105806105 |
-------------------------------------
-------------------------------------
| approxkl           | 0.004785531  |
| clipfrac           | 0.064453125  |
| explained_variance | 0.888        |
| fps                | 213          |
| n_updates          | 350          |
| policy_entropy     | 0.7739549    |
| policy_loss        | -0.011369865 |
| serial_timesteps   | 44800        |
| time_elapsed       | 850          |
| total_timesteps    | 179200       |
| value_loss         | 0.010883015  |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.0022490395 |
| clipfrac           | 0.019042969  |
| explained_variance | 0.98         |
| fps                | 226          |
| n_updates          | 510          |
| policy_entropy     | 0.8886798    |
| policy_loss        | -0.007988788 |
| serial_timesteps   | 65280        |
| time_elapsed       | 1.23e+03     |
| total_timesteps    | 261120       |
| value_loss         | 0.0020773949 |
-------------------------------------
-------------------------------------
| approxkl           | 0.0100695165 |
| clipfrac           | 0.12988281   |
| explained_variance | 0.939        |
| fps                | 222          |
| n_updates          | 520          |
| policy_entropy     | 0.8862252    |
| policy_loss        | -0.011079572 |
| serial_timesteps   | 66560        |
| time_elapsed       | 1.26e+03     |
| total_timesteps    | 266240       |
| value_loss         | 0.006312358  |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.0071912454 |
| clipfrac           | 0.09667969   |
| explained_variance | 0.961        |
| fps                | 222          |
| n_updates          | 680          |
| policy_entropy     | 0.6833181    |
| policy_loss        | -0.010240088 |
| serial_timesteps   | 87040        |
| time_elapsed       | 1.63e+03     |
| total_timesteps    | 348160       |
| value_loss         | 0.0030395747 |
-------------------------------------
-------------------------------------
| approxkl           | 0.006461937  |
| clipfrac           | 0.09326172   |
| explained_variance | 0.972        |
| fps                | 228          |
| n_updates          | 690          |
| policy_entropy     | 0.70786595   |
| policy_loss        | -0.014169835 |
| serial_timesteps   | 88320        |
| time_elapsed       | 1.65e+03     |
| total_timesteps    | 353280       |
| value_loss         | 0.0032674423 |
-------------------------------------
------------

...

-------------------------------------
| approxkl           | 0.019459281  |
| clipfrac           | 0.06933594   |
| explained_variance | 0.9          |
| fps                | 228          |
| n_updates          | 7140         |
| policy_entropy     | 0.2612937    |
| policy_loss        | -0.018558787 |
| serial_timesteps   | 913920       |
| time_elapsed       | 1.64e+04     |
| total_timesteps    | 3655680      |
| value_loss         | 0.023809213  |
-------------------------------------
-------------------------------------
| approxkl           | 0.039071415  |
| clipfrac           | 0.08154297   |
| explained_variance | 0.948        |
| fps                | 222          |
| n_updates          | 7150         |
| policy_entropy     | 0.24578619   |
| policy_loss        | -0.020492267 |
| serial_timesteps   | 915200       |
| time_elapsed       | 1.64e+04     |
| total_timesteps    | 3660800      |
| value_loss         | 0.010723526  |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.02368304   |
| clipfrac           | 0.095703125  |
| explained_variance | 0.948        |
| fps                | 227          |
| n_updates          | 7310         |
| policy_entropy     | 0.28351784   |
| policy_loss        | -0.025588369 |
| serial_timesteps   | 935680       |
| time_elapsed       | 1.68e+04     |
| total_timesteps    | 3742720      |
| value_loss         | 0.014612645  |
-------------------------------------
-------------------------------------
| approxkl           | 0.018036269  |
| clipfrac           | 0.10205078   |
| explained_variance | 0.917        |
| fps                | 234          |
| n_updates          | 7320         |
| policy_entropy     | 0.33600107   |
| policy_loss        | -0.029336331 |
| serial_timesteps   | 936960       |
| time_elapsed       | 1.68e+04     |
| total_timesteps    | 3747840      |
| value_loss         | 0.018549155  |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.024193347  |
| clipfrac           | 0.099609375  |
| explained_variance | 0.878        |
| fps                | 242          |
| n_updates          | 7480         |
| policy_entropy     | 0.27913964   |
| policy_loss        | -0.02744127  |
| serial_timesteps   | 957440       |
| time_elapsed       | 1.71e+04     |
| total_timesteps    | 3829760      |
| value_loss         | 0.0121323615 |
-------------------------------------
------------------------------------
| approxkl           | 0.013951942 |
| clipfrac           | 0.07861328  |
| explained_variance | 0.895       |
| fps                | 235         |
| n_updates          | 7490        |
| policy_entropy     | 0.2785622   |
| policy_loss        | -0.01978524 |
| serial_timesteps   | 958720      |
| time_elapsed       | 1.72e+04    |
| total_timesteps    | 3834880     |
| value_loss         | 0.025665332 |
------------------------------------
-------------------------

-------------------------------------
| approxkl           | 0.017825097  |
| clipfrac           | 0.08984375   |
| explained_variance | 0.874        |
| fps                | 235          |
| n_updates          | 7650         |
| policy_entropy     | 0.3382021    |
| policy_loss        | -0.025882285 |
| serial_timesteps   | 979200       |
| time_elapsed       | 1.75e+04     |
| total_timesteps    | 3916800      |
| value_loss         | 0.02314691   |
-------------------------------------
------------------------------------
| approxkl           | 0.024090612 |
| clipfrac           | 0.11035156  |
| explained_variance | 0.898       |
| fps                | 237         |
| n_updates          | 7660        |
| policy_entropy     | 0.28002086  |
| policy_loss        | -0.02339534 |
| serial_timesteps   | 980480      |
| time_elapsed       | 1.75e+04    |
| total_timesteps    | 3921920     |
| value_loss         | 0.011257232 |
------------------------------------
-------------------------

Tempo em horas: 4.967752947277493


In [14]:
lista_recompensas=[]
for i in range(5):
    obs=env.reset()
    terminou=False
    recompensa_episodio=0
    while terminou != True:
        acao, estado = modelo.predict(obs)
        obs, recompensa, terminou, info=env.step(acao)
        recompensa_episodio+=recompensa
    print('Recompensa episodio',i,':',recompensa_episodio)
    lista_recompensas.append(recompensa_episodio)
    
print('Recompensa media:',sum(lista_recompensas)/len(lista_recompensas))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [15]:
env = gym.make('BreakoutNoFrameskip-v4')
env = wrapper(env)

obs = env.reset()
recompensa_total = 0
terminou = False

while terminou!= True:
    env.render()
    time.sleep(0.02)
    acao, estado = modelo.predict(obs)
    obs, recompensa, terminou, info = env.step(acao)
    recompensa_total+=recompensa
print('Recompensa total:', recompensa_total)
env.close()

Recompensa total: 16.0
